**Author: PSI**\
**Modified by DLE**

**Project Descriptio:2018 LEHD Data QC (version 7.5) of the origin- destination (OD) data**

QC checks: <br>

Part 1-Compare the source data and staging tables for: <br>
-Field totals<br>
-Number of Records <br>
-Spot checking data <br>

Part 2-Checking outliers in data by sorting data from highest to lowest and visualizing those census tracts in ArcGISPro

**Installing Packages**


In [5]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc
import sys

**Loading the source data** 

In [2]:
path = r'R:\DPOE\LEHD LODES\7.5\Source\OD_Data\Unzipped_RD'

In [ ]:
%%time 

2016 source data
csv_2016 = glob.glob(path + "/*2016.csv")
list_2016 = []

for filename in csv_2016:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_2016.append(df)

od_2016 = pd.concat(list_2016, axis=0, ignore_index=True)

od_2016


In [3]:
%%time

# 2017 source data
csv_2017 = glob.glob(path + "/*2017.csv")
list_2017 = []

for filename in csv_2017:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_2017.append(df)

od_2017 = pd.concat(list_2017, axis=0, ignore_index=True)


Wall time: 3min 1s


In [6]:
sys.getsizeof(od_2017)

5808207024

In [7]:
od_2017.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55848143 entries, 0 to 55848142
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   w_geocode   float64
 1   h_geocode   float64
 2   S000        int64  
 3   SA01        int64  
 4   SA02        int64  
 5   SA03        int64  
 6   SE01        int64  
 7   SE02        int64  
 8   SE03        int64  
 9   SI01        int64  
 10  SI02        int64  
 11  SI03        int64  
 12  createdate  int64  
dtypes: float64(2), int64(11)
memory usage: 5.4 GB


In [ ]:
%%time

# 2018 source data
csv_2018 = glob.glob(path + "/*2018.csv")
list_2018 = []

for filename in csv_2018:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_2018.append(df)

od_2018 = pd.concat(list_2018, axis=0, ignore_index=True)

**Loading the staging tables** 

In [8]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=socioec_data;'
                      'Trusted_Connection=yes;')

In [ ]:
%%time

#2016
od_db16 ="SELECT * "\
                  "FROM [socioec_data].[lehd_lodes].[od_7_5]"\
                   "WHERE yr= 2016"
db16 = pd.read_sql_query(od_db16, conn)

In [9]:
%%time

#2017
od_db17 ="SELECT * "\
                  "FROM [socioec_data].[lehd_lodes].[od_7_5]"\
                   "WHERE yr= 2017"
db17 = pd.read_sql_query(od_db17, conn)

Wall time: 4min 46s


In [ ]:
%%time

#2018
od_db18 ="SELECT * "\
                  "FROM [socioec_data].[lehd_lodes].[od_7_5]"\
                   "WHERE yr= 2018"
db18 = pd.read_sql_query(od_db18, conn)

In [10]:
db17

,w_geoid,h_geoid,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,type,yr
0,060730085111106,060375734023016,1,0,1,0,0,0,1,0,1,0,JT00,2017
1,060730085111106,060376035002010,1,0,1,0,0,0,1,0,0,1,JT00,2017
2,060730085111106,060379012051013,1,0,1,0,0,0,1,0,0,1,JT00,2017
3,060730085111106,060379012051019,1,0,0,1,0,1,0,1,0,0,JT00,2017
4,060730085111106,060379012054094,1,0,0,1,0,0,1,0,0,1,JT00,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5581346,060730083052000,060730134144000,1,0,1,0,0,0,1,0,0,1,JT00,2017
5581347,060730083052000,060730134144003,1,0,0,1,0,0,1,0,0,1,JT00,2017
5581348,060730083052000,060730134144005,1,0,1,0,0,1,0,0,0,1,JT00,2017
5581349,060730083052000,060730134151007,1,0,1,0,0,0,1,0,0,1,JT00,2017


### QC checks
1. Compare the sum totals by columns
2. Compare data types 
3. Conduct spot checks for top 1000 rows<br> 
**Note**- The source data will have to be subsetted to include only San Diego geoid (starting with 6073) and remove unnecessary columns 

In [ ]:
%%time
# Subsetting the source data 
od_2016_sub= od_2016 [od_2016 ['w_geocode'].astype(str).str.startswith('6073') | od_2016 ['h_geocode'].astype(str).str.startswith('6073') ]

In [12]:
%%time
od_2017_sub= od_2017 [od_2017 ['w_geocode'].astype(str).str.startswith('6073') | od_2017 ['h_geocode'].astype(str).str.startswith('6073') ]

Wall time: 2min 31s


In [ ]:
%%time
od_2018_sub= od_2018 [od_2018 ['w_geocode'].astype(str).str.startswith('6073') | od_2018 ['h_geocode'].astype(str).str.startswith('6073') ]

In [ ]:
# od_2016_sub
od_2017_sub
# od_2018_sub

In [21]:
# print('2016: {0} {1}'.format(len(od_2016_sub), len(db16)))
print('2017: {0} {1} ({2})'.format(len(od_2017_sub), len(db17), len(od_2017_sub)-len(db17)))
# print('2018: {0} {1}'.format(len(od_2018_sub), len(db18)))

2017: 5593837 5581351 (12486)


In [ ]:
# od_2017.info(memory_usage='deep')

### Compare data types

In [13]:
db17.dtypes

w_geoid    object
h_geoid    object
S000        int64
SA01        int64
SA02        int64
SA03        int64
SE01        int64
SE02        int64
SE03        int64
SI01        int64
SI02        int64
SI03        int64
type       object
yr          int64
dtype: object

In [14]:
od_2017_sub.dtypes

w_geocode     float64
h_geocode     float64
S000            int64
SA01            int64
SA02            int64
SA03            int64
SE01            int64
SE02            int64
SE03            int64
SI01            int64
SI02            int64
SI03            int64
createdate      int64
dtype: object

### Compare sum totals

In [18]:
%%time

# DB - remove descriptive fields
db17_sum = db17.iloc[: , 2:-2]

total = db17_sum.sum()
total.name = 'Total'

db17_sum = db17_sum.append(total.transpose())
db17_sum

Wall time: 2.59 s


,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
0,1,0,1,0,0,0,1,0,1,0
1,1,0,1,0,0,0,1,0,0,1
2,1,0,1,0,0,0,1,0,0,1
3,1,0,0,1,0,1,0,1,0,0
4,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
5581347,1,0,0,1,0,0,1,0,0,1
5581348,1,0,1,0,0,1,0,0,0,1
5581349,1,0,1,0,0,0,1,0,0,1
5581350,1,0,1,0,0,0,1,0,0,1


In [20]:
%%time

# CSV - remove descriptive fields
od17_sum = od_2017_sub.iloc[:, 2:-1]

total = od17_sum.sum()
total.name = 'Total'

od17_sum = od17_sum.append(total.transpose())
od17_sum

Wall time: 1.9 s


,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
115272,1,1,0,0,1,0,0,1,0,0
115273,1,0,1,0,0,0,1,0,0,1
115274,1,0,1,0,0,0,1,0,0,1
115275,1,0,0,1,0,1,0,0,0,1
115276,1,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
55844505,1,0,1,0,0,0,1,0,0,1
55846505,1,0,1,0,0,0,1,0,0,1
55847787,1,0,0,1,0,0,1,0,0,1
55847902,1,0,1,0,0,0,1,0,0,1


### Check Census Blocks included

In [15]:
## CHECKING DB TABLE

# h_geoid blocks
qry = "SELECT DISTINCT h_geoid FROM socioec_data.lehd_lodes.od_7_5 WHERE yr = 2017"
db_blocks = pd.read_sql_query(qry, conn)
db_count_h = len(db_blocks)
print('h_geoid: {0}'.format(db_count_h))

# w_geoid blocks
qry = "SELECT DISTINCT w_geoid FROM socioec_data.lehd_lodes.od_7_5 WHERE yr = 2017"
db_blocks = pd.read_sql_query(qry, conn)
db_count_w = len(db_blocks)
print('w_geoid: {0}'.format(db_count_w))

h_geoid: 152472
w_geoid: 59724


In [16]:
## CHECKING CSV

# h_geocode blocks
od_blocks = od_2017_sub.drop_duplicates(subset=['h_geocode'])
od_count_h = len(od_blocks)
print('h_geocode: {0}'.format(od_count_h))

# w_geocode blocks
od_blocks = od_2017_sub.drop_duplicates(subset=['w_geocode'])
od_count_w = len(od_blocks)
print('w_geocode: {0}'.format(od_count_w))

h_geocode: 153964
w_geocode: 59727


In [17]:
print('h difference: {0}'.format(od_count_h - db_count_h))
print('w difference: {0}'.format(od_count_w - db_count_w))

h difference: 1492
w difference: 3
